In [ ]:
# 📌 Setup and Imports
import cv2
import numpy as np
import joblib
import base64
import io
import requests
import supabase
import time
import threading
from flask import Flask, request, jsonify
from flask_cors import CORS
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from PIL import Image
import nest_asyncio

# 📌 Allow Flask to run inside Jupyter Notebook
nest_asyncio.apply()

# 📌 Flask App
app = Flask(__name__)
CORS(app, resources={r"/api/*": {"origins": "*"}})

# 📌 Supabase Initialization
SUPABASE_URL = "" #replace with you URL
SUPABASE_KEY = "" # replace with your key
supabase_client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)

# 📌 Load Models
try:
    svm_fresh_rotten = joblib.load("models/fresh_vs_rotten_svm_model.pkl")
    svm_food_type = joblib.load("models/food_type_svm_model.pkl")
    food_categories = joblib.load("models/food_categories.pkl")
except Exception as e:
    print(f"❌ Error loading models: {e}")
    raise

# 📌 Feature Extraction Model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(220, 220, 3))
feature_model = Model(inputs=base_model.input, outputs=base_model.output)

# 📌 Webcam URL
IP_WEBCAM_URL = "http://192.168.114.28:8080/shot.jpg"

# 📌 Fetch Image Frame from IP Webcam
def fetch_ip_webcam_frame():
    try:
        response = requests.get(IP_WEBCAM_URL, timeout=2)
        if response.status_code == 200:
            img_array = np.array(Image.open(io.BytesIO(response.content)))
            return img_array
        else:
            return None
    except Exception as e:
        print(f"❌ Error fetching image: {e}")
        return None

# 📌 Prediction Logic
def predict_food(img):
    try:
        img_array = cv2.resize(img, (220, 220))
        img_array = img_array.astype("float32")
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)

        features = feature_model.predict(img_array)
        features = features.flatten().reshape(1, -1)

        freshness_prediction = svm_fresh_rotten.predict(features)[0]
        food_type_prediction = svm_food_type.predict(features)[0]
        food_category = food_categories[food_type_prediction]

        return {
            #"freshness": "Fresh" if freshness_prediction == 1 else "Rotten",
            "food_type": food_category
        }
    except Exception as e:
        return {"error": f"Error processing image: {e}"}

# 📌 Update or Insert to Supabase
def update_or_insert_food(food_name):
    try:
        existing_item = supabase_client.from_("fridge_contents").select("*").eq("name", food_name).execute()

        if existing_item.data:
            item_id = existing_item.data[0]["id"]
            current_quantity = existing_item.data[0]["quantity"]

            supabase_client.from_("fridge_contents").update({
                "quantity": current_quantity + 1
            }).eq("id", item_id).execute()
        else:
            supabase_client.from_("fridge_contents").insert([{
                "name": food_name,
                "quantity": 1,
                "storage_day": 1
            }]).execute()

            print(f"✅ Inserted new item: {food_name} with storage_day = 1")
    except Exception as e:
        print(f"❌ DB Update Error: {e}")

# 📌 Increment Storage Day Thread
def increment_storage_days():
    while True:
        try:
            response = supabase_client.from_("fridge_contents").select("id, storage_day").execute()
            if response.data:
                for item in response.data:
                    item_id = item["id"]
                    new_storage_day = item["storage_day"] + 1

                    supabase_client.from_("fridge_contents").update(
                        {"storage_day": new_storage_day}
                    ).eq("id", item_id).execute()
            print("✅ Storage days incremented.")
        except Exception as e:
            print(f"❌ Error incrementing storage_day: {e}")
        time.sleep(6000)

# 📌 Start Increment Thread
threading.Thread(target=increment_storage_days, daemon=True).start()

# 📌 Flask Routes
@app.route("/api/analyze_image", methods=["POST"])
def analyze_image():
    try:
        image = fetch_ip_webcam_frame()
        if image is None:
            return jsonify({"error": "Failed to fetch image from IP Webcam"}), 500

        # Ensure RGB conversion
        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif image.shape[2] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

        result = predict_food(image)

        # Only update DB if fresh
        if "error" not in result and result["food_type"].startswith('Fresh'):
            update_or_insert_food(result["food_type"])
        elif "error" not in result:
            print(f"🚫 Rotten item detected: {result['food_type']} — not inserted.")

        return jsonify(result)
    except Exception as e:
        return jsonify({"error": f"Exception analyzing image: {e}"}), 500

@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "API is running!"})

# 📌 Run Flask in Jupyter
from werkzeug.serving import run_simple
run_simple("0.0.0.0", 5001, app, use_reloader=False, use_debugger=True)


c:\Users\venka\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


❌ Error incrementing storage_day: [Errno 11001] getaddrinfo failed


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.114.213:5001
Press CTRL+C to quit
127.0.0.1 - - [29/Apr/2025 22:01:10] "OPTIONS /api/analyze_image HTTP/1.1" 200 -


1/1 [==============================] - 11s 11s/step


127.0.0.1 - - [29/Apr/2025 22:01:21] "POST /api/analyze_image HTTP/1.1" 200 -


...

🚫 Rotten item detected: RottenCarrot — not inserted.
1/1 [==============================] - 0s 19ms/step


127.0.0.1 - - [29/Apr/2025 22:07:04] "POST /api/analyze_image HTTP/1.1" 200 -


❌ DB Update Error: {'message': 'JSON could not be generated', 'code': 503, 'hint': 'Refer to full message for details', 'details': "b'upstream connect error or disconnect/reset before headers. retried and the latest reset reason: remote connection failure, transport failure reason: delayed connect error: 111'"}
1/1 [==============================] - 0s 20ms/step


127.0.0.1 - - [29/Apr/2025 22:07:04] "POST /api/analyze_image HTTP/1.1" 200 -


🚫 Rotten item detected: RottenCarrot — not inserted.
